In [1]:
import numpy as np
import cs_vqe as c
import ast
import os

from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
import scipy as sp

In [2]:
from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [3]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [4]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [5]:
# mol_key = 'H2_6-31G_singlet'  
# mol_key ='H2-O1_STO-3G_singlet'
# mol_key = 'H1-He1_3-21G_singlet_1+'
mol_key = 'H3_STO-3G_singlet_1+'

# currently index 2 is contextual part
# ''''''''''''''''3 is NON contextual part

# join together for full Hamiltonian:
ham = hamiltonians[mol_key][2]
ham.update(hamiltonians[mol_key][3]) # full H 
ham

{'III': -1.7512307459285525,
 'IIX': -0.023568152336180023,
 'IIZ': -0.017109477140260287,
 'IXI': 0.02356815233618002,
 'IXZ': 0.02356815233617983,
 'IYY': -0.07195737217001562,
 'IZI': -0.017109477140260287,
 'IZX': -0.023568152336179825,
 'IZZ': 0.31270210682950855,
 'XII': 0.01872992170537467,
 'XIX': 0.023568139980123585,
 'XIZ': 0.01872992170537467,
 'XXI': -0.023568139980123585,
 'XXX': 0.03597868636603963,
 'XXZ': -0.023568139980123585,
 'XYY': -0.03597868636603963,
 'XZI': 0.01872992170537467,
 'XZX': 0.023568139980123585,
 'XZZ': 0.01872992170537467,
 'YIY': 0.023568139980123585,
 'YXY': 0.03597868636603963,
 'YYI': -0.023568139980123585,
 'YYX': 0.03597868636603963,
 'YYZ': -0.023568139980123585,
 'YZY': 0.023568139980123585,
 'ZII': -0.45436486525596403,
 'ZIX': -0.023568152336180023,
 'ZIZ': 0.37110605476609787,
 'ZXI': 0.02356815233618002,
 'ZXZ': 0.02356815233617983,
 'ZYY': -0.07195737217001562,
 'ZZI': 0.37110605476609804,
 'ZZX': -0.023568152336179825,
 'ZZZ': -0.2878

In [6]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  3


# Get non-contextual H

In [7]:
nonH_guesses = c.greedy_dfs(ham, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg

Split into:

$$H = H_{c} + H_{nc}$$

In [8]:
nonCon_H = {}
Con_H = {}

for P in ham:
    if P in nonH:
        nonCon_H[P]=ham[P]
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [9]:
print('Is NONcontextual correct:', not c.contextualQ_ham(nonCon_H))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [10]:
bool_flag, Z_list, T_list = c.contextualQ(list(nonCon_H.keys()), verbose=True)

In [11]:
Z_list

['III', 'IZZ', 'ZIZ', 'ZZI']

In [12]:
T_list

['IIZ', 'IZI', 'XXX', 'XYY', 'YXY', 'YYX', 'ZII', 'ZZZ']

## Get quasi model

First we define

- $C_{i1}$ = first Pauli in each $C_{i}$ set
- $A_{ij} = C_{ij}C_{1i}$


- $G^{prime} = \{1 P_{i} \;| \; i=1,2,...,|Z| \}$
    - aka all the completely commuting terms with coefficients set to +1!

- We define G to be an independent set of $G^{prime}$
    - where $G \subseteq G^{prime}$


In [13]:
G_list, Ci1_list, all_mappings = c.quasi_model(nonCon_H)

In [14]:
print('non-independent Z list:', Z_list)
print('G (independent) Z list:', G_list)

non-independent Z list: ['III', 'IZZ', 'ZIZ', 'ZZI']
G (independent) Z list: ['ZIZ', 'IZZ']


In [15]:
print('all Ci1 terms:', Ci1_list)

all Ci1 terms: ['IIZ', 'XXX']


$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

In [16]:
# Assemble all the mappings from terms in the Hamiltonian to their products in R:
all_mappings

{'III': [[], [], 1],
 'IIZ': [[], ['IIZ'], 1],
 'IZI': [['IZZ'], ['IIZ'], 1],
 'IZZ': [['IZZ'], [], 1],
 'XXX': [[], ['XXX'], 1],
 'XYY': [['IZZ'], ['XXX'], (-1+0j)],
 'YXY': [['ZIZ'], ['XXX'], (-1+0j)],
 'YYX': [['ZIZ', 'IZZ'], ['XXX'], (-1+0j)],
 'ZII': [['ZIZ'], ['IIZ'], 1],
 'ZIZ': [['ZIZ'], [], 1],
 'ZZI': [['ZIZ', 'IZZ'], [], 1],
 'ZZZ': [['ZIZ', 'IZZ'], ['IIZ'], 1]}

Overall $R$ is basically reduced non-contextual set
- where everything in original non-contextual set can be found by **inference!**

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [17]:
model = [G_list, Ci1_list, all_mappings]

fn_form = c.energy_function_form(nonCon_H, model)

# returns [
#            denstion of q,
#            dimension of r,
#            [coeff, indices of q's, indices of r's, term in Hamiltonian]
#         ]


In [18]:
fn_form

[2,
 2,
 [[-1.7512307459285525, [], [], 'III'],
  [-0.017109477140260287, [], [0], 'IIZ'],
  [-0.017109477140260287, [1], [0], 'IZI'],
  [0.31270210682950855, [1], [], 'IZZ'],
  [0.03597868636603963, [], [1], 'XXX'],
  [(0.03597868636603963-0j), [1], [1], 'XYY'],
  [(-0.03597868636603963+0j), [0], [1], 'YXY'],
  [(-0.03597868636603963+0j), [0, 1], [1], 'YYX'],
  [-0.45436486525596403, [0], [0], 'ZII'],
  [0.37110605476609787, [0], [], 'ZIZ'],
  [0.37110605476609804, [0, 1], [], 'ZZI'],
  [-0.2878474382772282, [0, 1], [0], 'ZZZ']]]

In [19]:
Energy_function = c.energy_function(fn_form)

In [20]:
import random

### now for the q terms we only have +1 or -1 assignment!
q_variables = [random.choice([1,-1]) for _ in range(fn_form[0])]


### r variables is anything that makes up unit vector!
r_variables = c.angular(np.arange(0,2*np.pi, fn_form[1]))
r_variables

(1.0, -0.0, -0.0, -0.0, 0.0)

In [21]:
 Energy_function(*q_variables,*r_variables)

-1.897415425779325

find_gs_nonconfunction optimizes above steps by:
1. brute forcing all choices of ```q_variables```
    - ```itertools.product([1,-1],repeat=fn_form[0])```
2. optimizing over ```r_variables``` (in code ```x```)
    - using SciPy optimizer!

In [22]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-2.903212918716725
[[-1, 1], [-0.9799593376325533, -0.19919763198082302]]


In [23]:
## check
Energy_function(*ground_state_params[0],*ground_state_params[1]) == lowest_eigenvalue

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [24]:
ground_state_params

[[-1, 1], [-0.9799593376325533, -0.19919763198082302]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [25]:
model = [G_list, Ci1_list, all_mappings]

print(G_list) # G_j terms!
print(Ci1_list) # mathcal(A)

['ZIZ', 'IZZ']
['IIZ', 'XXX']


to SINGLE QUBIT pauli Z operators!

- to map the operators in $G$ to single qubit Pauli operators, we use $\frac{\pi}{2}$ rotations!

- note $\mathcal{A}$ is an anti-commuting set... therefore we can use $N-1$ rotations as in unitary partitioning's sequence of rotations to do this!
    - $R^{\dagger}\mathcal{A} R = \text{single Pauli op}$

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

function ```diagonalize_epistemic```:
1. first if else statement:
    - if cliques present:
        - first maps A to single Pauli operator (if cliques present)
        - then rotates to diagonlize G union with single Pauli opator of A (hence GuA name!)
    - else if NO cliques present:
        - gets rotations to diagonlize G
        
     - these rotations make up GuA term in code!
2. NEXT code loops over terms in GuA (denoted as g in code)
    - if g is not a single qubit $Z$:
        - code generates code to rotate operator to make g diagonal (rotations)
        - then constructs map of g to single Z (J rotation)
    - Note R is applied to GuA
    
    
#########
- Note rotations are given in Appendix A of https://arxiv.org/pdf/2011.10027.pdf
    - First code checks if g op in GuA is diagonal
        - if so then needs to apply "K" rotation (involving $Y$ and $I$ operators (see pg 11 top) to make it NOT diagononal
    - now operator will be diagnoal!
    - next generate "J" rotation
        - turns non-diagonal operator into a single qubit $Z$ operator!

In [26]:
# Get sequence of rotations requried to diagonalize the generators for the noncontextual ground state!

Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops = c.diagonalize_epistemic(model,
                                                                                                            fn_form,
                                                                                                            ground_state_params)

In [27]:
# rotations to map A to single Pauli operator!
Rotations_list

[[3.342131729291596, 'XXY'],
 ['pi/2', 'YII'],
 ['pi/2', 'YIZ'],
 ['pi/2', 'IYI'],
 ['pi/2', 'IYZ']]

In [28]:
# rotations to diagonlize G
diagonalized_generators_GuA

['ZII', 'IZI', 'IIZ']

In [29]:
eigen_vals_nonC_ground_state_GuA_ops

array([ 1., -1.,  1.])

In [30]:
# for rotation in Rotations_list:
#     for p in ham:
#         if not c.commute(rotation[1],p):
#             print(rotation)
#     print()

In [31]:
model = [G_list, Ci1_list, all_mappings]
ham_noncon = hamiltonians[mol_key][3]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(
               ham_noncon,#nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

# Will's method

In [32]:
ham_noncon = hamiltonians[mol_key][3]
n_qubits = hamiltonians[mol_key][1]
true_gs= hamiltonians[mol_key][4]

true_gs, approxs_out, errors_out, order_out = c.csvqe_approximations_heuristic(ham,
                                                                                   ham_noncon,
                                                                                   n_qubits, 
                                                                                   true_gs)
order_out

[0, 2, 1]

In [33]:
errors_out

[0.012805571551730122,
 0.0003080609044734395,
 -1.3322676295501878e-15,
 8.881784197001252e-16]

In [34]:
### old way
# order = list(range(hamiltonians[mol_key][1]))
order = order_out
red_H = c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           order)
len(red_H)

4

In [35]:
list(range(hamiltonians[mol_key][1]))

[0, 1, 2]

In [36]:
red_H[2]

{'II': -1.438528639099044,
 'IZ': 0.01919944551942593,
 'ZX': 0.07733163398184689,
 'YY': -0.06305338666771393,
 'XI': 0.051042861476446974,
 'XZ': -0.03745984341074934,
 'IX': -0.07733163397053659,
 'ZZ': -0.7416716156049102,
 'ZI': -0.7422121095321959,
 'XX': 0.07195737217001562}

In [37]:
approxs_out

[-2.903212918716724,
 -2.9157104293639806,
 -2.9160184902684554,
 -2.916018490268453]

# MY way

In [38]:
import cs_vqe_MY_CHANGES as c_new

In [39]:
ham_noncon = hamiltonians[mol_key][3]
n_qubits = hamiltonians[mol_key][1]
true_gs= hamiltonians[mol_key][4]

true_gs, approxs_out, errors_out, order_out = c_new.csvqe_approximations_heuristic(ham,
                                                                                   ham_noncon,
                                                                                   n_qubits, 
                                                                                   true_gs)

from copy import deepcopy 
updated_order = deepcopy(order_out)
updated_order

[0, 2, 1]

In [40]:
errors_out

[0.012805571551729678, 0.0003080609044729954, 0.0, 2.6645352591003757e-15]

In [41]:

red_H_SeqRot = c_new.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           updated_order,
                           n_qubits,
                          check_reduction=True)#order)
len(red_H_SeqRot)

4

In [42]:
red_H_SeqRot[2]

{'II': (-1.4385286390990442+0j),
 'IZ': (0.019199445519425935+0j),
 'ZX': (0.0773316339818469+0j),
 'YY': (-0.06305338666771394+0j),
 'XI': (0.05104286147644698+0j),
 'XZ': (-0.037459843410749344+0j),
 'IX': (-0.07733163397053659+0j),
 'ZZ': (-0.7416716156049104+0j),
 'ZI': (-0.7422121095321959+0j),
 'XX': (0.07195737217001562+0j)}

In [49]:
red_H == red_H_SeqRot

False

## Check energies

In [50]:
hamilt_ind = 0
# red_H[hamilt_ind]

In [51]:
import conversion_scripts as conv_scr

H = conv_scr.Get_Openfermion_Hamiltonian(red_H[hamilt_ind])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
# sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
min(np.linalg.eigvalsh(sparseH.todense()))

-2.903212918716725

In [52]:
H = conv_scr.Get_Openfermion_Hamiltonian(red_H_SeqRot[hamilt_ind])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
# sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
min(np.linalg.eigvalsh(sparseH.todense()))

-2.903212918716725

In [53]:
H = conv_scr.Get_Openfermion_Hamiltonian(ham)
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
# sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
min(np.linalg.eigvalsh(sparseH.todense()))

-2.916018490268456

In [48]:
fdsd

NameError: name 'fdsd' is not defined

In [ ]:
import cs_vqe_with_LCU as c_LCU

In [ ]:
ham_noncon = hamiltonians[mol_key][3]
n_qubits = hamiltonians[mol_key][1]
true_gs= hamiltonians[mol_key][4]
N_index=0
gs_true, approxs_out, errors_out, order_out = c_LCU.csvqe_approximations_heuristic_LCU(ham, 
                                                                                       ham_noncon, 
                                                                                       n_qubits,
                                                                                       true_gs,
                                                                                       N_index,
                                                                                       check_reduction=True)

from copy import deepcopy 
updated_order = deepcopy(order_out)
updated_order

In [ ]:
approxs_out

In [ ]:
nonCon_H == hamiltonians[mol_key][3] # something odd here!

In [ ]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

In [ ]:

red_H_LCU = c_LCU.get_reduced_hamiltonians_LCU(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           updated_order,
                           n_qubits,
                          0,
                          check_reduction=True)#order)
len(red_H_LCU)

In [ ]:
H = conv_scr.Get_Openfermion_Hamiltonian(red_H_LCU[1])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
# sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
min(np.linalg.eigvalsh(sparseH.todense()))

In [ ]:
red_H_LCU == red_H_SeqRot # true when same order used!

In [ ]:
import cirq

In [ ]:
HSX = cirq.H._unitary_() @ cirq.S._unitary_() @ cirq.X._unitary_()

Y = cirq.Y._unitary_()

In [ ]:
HSX @ Y @ HSX.conj().T

In [ ]:
HS = cirq.H._unitary_() @ (cirq.S**-1)._unitary_() 

Y = cirq.Y._unitary_()

HS @ Y @ HS.conj().T

In [ ]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

In [ ]:
rotations = []
ep_state = ground_state_params
# if there are cliques...
if fn_form[1] > 0:
    # rotations to map A to a single Pauli (to be applied on left)
    for i in range(1,fn_form[1]):
        theta = np.arctan2(ep_state[1][i],np.sqrt(sum([ep_state[1][j]**2 for j in range(i)])))
        if i == 1 and ep_state[1][0] < 0:
            theta = np.pi - theta
        generator = c.pauli_mult(model[1][0],model[1][i])
        sgn = generator[1].imag
        rotations.append( [sgn*theta, generator[0]] )
rotations

In [ ]:
# We now have non contextual ground state: $(\vec{q}, \vec{r})$

ep_state[1] # this is r_Vec (coefficients for sciprt A)

In [ ]:
Ci1_list # or script_A

In [ ]:
1j*np.sin(3.342131729291596)

In [ ]:
c.apply_rotation(rotations[0], Ci1_list[1])

In [ ]:
conj_rot = [rotations[0][0]*-1, rotations[0][1]]
conj_rot

In [ ]:
c.apply_rotation(conj_rot, 'XXX')

In [ ]:
c.apply_rotation(conj_rot, 'IIZ')

In [ ]:
A_op2 * 

In [ ]:
A_op1 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(Ci1_list[0]) if Pstr!='I'])
A_op2 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(Ci1_list[1]) if Pstr!='I'])

R_op = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(rotations[0][1]) if Pstr!='I'])


In [ ]:
from openfermion import QubitOperator, hermitian_conjugated
script_A = QubitOperator(A_op1, ep_state[1][0]) +  QubitOperator(A_op2, ep_state[1][1])
R = QubitOperator('', np.cos(rotations[0][0]/2)) + QubitOperator(R_op, 1j*np.sin(rotations[0][0]/2)) 

In [ ]:
R * script_A * hermitian_conjugated(R)

In [ ]:
R * H 

In [ ]:
from openfermion import QubitOperator, hermitian_conjugated

# NOW divided theta by 2
R_my_method = QubitOperator('', np.cos(rotations[0][0]/2)) + QubitOperator('X0 X1 Y2', 1j*np.sin(rotations[0][0]/2)) 

In [ ]:
R_my_method* script_A * hermitian_conjugated(R_my_method)

In [ ]:
R_my_method

In [ ]:
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import Get_Xsk_op_list
S_index = 0
N_Qubits=3
check_reduction=True

X_sk_theta_sk_list, normalised_FULL_set, Ps, gamma_l = Get_Xsk_op_list(list(script_A), 
                                                                       S_index, 
                                                                       N_Qubits,
                                                                       check_reduction=False,
                                                                       atol=1e-8,
                                                                       rtol=1e-05)

In [ ]:
from functools import reduce

R_sk_OP_list = []
for X_sk_Op, theta_sk in X_sk_theta_sk_list:
    op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
    R_sk_OP_list.append(op)

R_S_op = reduce(lambda x,y: x*y, R_sk_OP_list[::-1])  # <- note reverse order!
R_S_op

In [ ]:
from openfermion import hermitian_conjugated
R_S_dag = hermitian_conjugated(R_S_op)
R_S_dag

In [ ]:
R_S_op * H * R_S_dag

In [ ]:
script_A

In [ ]:
A_op1 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(AC_set[0]) if Pstr!='I'])
A_op2 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(AC_set[1]) if Pstr!='I'])
A_op3 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(AC_set[2]) if Pstr!='I'])
A_op4 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(AC_set[3]) if Pstr!='I'])

R_op1 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(rotations[0][1]) if Pstr!='I'])
R_op2 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(rotations[1][1]) if Pstr!='I'])
R_op3 = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(rotations[2][1]) if Pstr!='I'])


# script_A = (QubitOperator(A_op1, ep_state[0]) +  QubitOperator(A_op2, ep_state[1]) + 
#             QubitOperator(A_op3, ep_state[2]) +  QubitOperator(A_op4, ep_state[3]) )


script_A = (QubitOperator(A_op1, 1) +  QubitOperator(A_op2, 1) + 
            QubitOperator(A_op3,1) +  QubitOperator(A_op4,1) )

R = (QubitOperator('', np.cos(rotations[0][0]/2)) + QubitOperator(R_op1, 1j*np.sin(rotations[0][0]/2)) + 
     QubitOperator('', np.cos(rotations[1][0]/2)) + QubitOperator(R_op2, 1j*np.sin(rotations[1][0]/2)) + 
     QubitOperator('', np.cos(rotations[2][0]/2)) + QubitOperator(R_op3, 1j*np.sin(rotations[2][0]/2)) 
    )

R * script_A * hermitian_conjugated(R)

In [ ]:
AC_set

In [ ]:
rotations = []
ep_state = [np.sqrt(0.2), -np.sqrt(0.3), np.sqrt(0.1), np.sqrt(0.4)]

AC_set = ['IXX', 'IXY', 'IXZ', 'XZI']

# if there are cliques...
if fn_form[1] > 0:
    # rotations to map A to a single Pauli (to be applied on left)
    for i in range(1,len(AC_set)):
        theta = np.arctan2(ep_state[i],np.sqrt(sum([ep_state[j]**2 for j in range(i)])))
        if i == 1 and ep_state[0] < 0:
            theta = np.pi - theta
        generator = c.pauli_mult(AC_set[0],AC_set[i])
        sgn = generator[1].imag
        rotations.append( [sgn*theta, generator[0]] )
rotations

In [ ]:
ham_rotated = dict(zip(AC_set,ep_state))

for r in rotations: # rotate the full Hamiltonian to the basis with diagonal noncontextual generators
#     print()
#     print(ham_rotated)
    ham_next = {}
    for t in ham_rotated.keys():
        t_set_next = c.apply_rotation(r,t)
#         print(t_set_next)
        for t_next in t_set_next.keys():
            if t_next in ham_next.keys():
                ham_next[t_next] = ham_next[t_next] + t_set_next[t_next]*ham_rotated[t]
            else:
                ham_next[t_next] = t_set_next[t_next]*ham_rotated[t]
    ham_rotated = deepcopy(ham_next)

ham_rotated

In [ ]:
R=[]
# R=QubitOperator()
for angle, op_str in rotations:
    R_op = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(op_str) if Pstr!='I'])
    qOp = QubitOperator('', np.cos(angle/2)) + QubitOperator(R_op, 1j*np.sin(angle/2)) 
    R.append(qOp)
#     R+= qOp

R

In [ ]:
Script_A = []
# Script_A =QubitOperator()
for term, coeff in zip(AC_set,ep_state):
    Pword = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(term) if Pstr!='I'])
    op = QubitOperator(Pword,coeff)
    Script_A.append(op)
#     Script_A+=op
    
Script_A

In [ ]:
rot_H = deepcopy(Script_A)
for rot in R:
    H_next = QubitOperator()
    for t in rot_H:
#         if t*rot == rot*t:
#             t_set_next = rot * t * hermitian_conjugated(rot)
#         else:
#             t_set_next = rot * t 
        t_set_next = rot * t * hermitian_conjugated(rot)
        H_next+=t_set_next
    rot_H = deepcopy(list(H_next))
rot_H

In [ ]:
ham_rotated = dict(zip(AC_set,ep_state))

for r in rotations: # rotate the full Hamiltonian to the basis with diagonal noncontextual generators
    ham_next = {}
    for t in ham_rotated.keys():
        t_set_next = c.apply_rotation(r,t)
        for t_next in t_set_next.keys():
            if t_next in ham_next.keys():
                ham_next[t_next] = ham_next[t_next] + t_set_next[t_next]*ham_rotated[t]
            else:
                ham_next[t_next] = t_set_next[t_next]*ham_rotated[t]
    ham_rotated = deepcopy(ham_next)

ham_rotated

In [ ]:
# test = [QubitOperator(*list(op.terms.keys()), 1) for op in Script_A]


# test = [R[0]*test[0], R[0]*test[1], R[0]*test[2], R[0]*test[3]]
# test = [R[1]*test[0], R[1]*test[1], R[1]*test[2], R[1]*test[3]]
# test = [R[2]*test[0], R[2]*test[1], R[2]*test[2], R[2]*test[3]]
# reduce(lambda x,y: x+y, test)

In [ ]:
one = R[0]*Script_A# * hermitian_conjugated(R[0])
# two = R[1]*one* hermitian_conjugated(R[1])
# three = R[2]*two* hermitian_conjugated(R[2])
# three
one

In [ ]:
[-0.8860771237926137, 'IIZ']
{'IXX': 0.7071067811865475, 'IXY': 0.0, 'IXZ': 0.31622776601683794, 'XZI': 0.6324555320336759}

In [ ]:
ham_rotated = dict(zip(AC_set,ep_state))

c.apply_rotation([-0.8860771237926137, 'IIZ'], list(ham_rotated.keys())[0])

In [ ]:
q = c.pauli_mult('IIZ',  list(ham_rotated.keys())[0])
q

In [ ]:
q[0] 

In [ ]:
R[0]*Script_A[0] #* hermitian_conjugated(R[0])

# R[0] * QubitOperator(*list(Script_A[0].terms.keys()), 1)

In [ ]:
R[0]

In [ ]:
Script_A[0]

In [ ]:
H_rot =  deepcopy(Script_A)
H_rot[0]

In [ ]:
H_rot =  deepcopy(Script_A)
# H_rot = reduce(lambda x,y: x+y, Script_A)
for rot in R:
#     H_next= QubitOperator()
    H_next=[]
    for term in H_rot:
        print(term)
        print()
        t = QubitOperator(*list(term.terms.keys()), 1)
        H_next.append(rot*t)
#         H_next+= rot*term #* hermitian_conjugated(rot)
    H_rot = deepcopy(reduce(lambda x,y: x+y, H_next))
reduce(lambda x,y: x+y, H_rot)

In [ ]:
ham_rotated = dict(zip(AC_set,ep_state))

for r in rotations: # rotate the full Hamiltonian to the basis with diagonal noncontextual generators
    print(r)
    ham_next = {}
    for t in ham_rotated.keys():
        t_set_next = c.apply_rotation(r,t)
        for t_next in t_set_next.keys():
            if t_next in ham_next.keys():
                ham_next[t_next] = ham_next[t_next] + t_set_next[t_next]*ham_rotated[t]
            else:
                ham_next[t_next] = t_set_next[t_next]*ham_rotated[t]
    
    print(ham_next)
    print()
    ham_rotated = deepcopy(ham_next)

ham_rotated

In [ ]:
X_sk_theta_sk_list

In [ ]:
R=[]
# R=QubitOperator()
for angle, op_str in rotations:
    R_op = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(op_str) if Pstr!='I'])
    qOp = QubitOperator('', np.cos(angle/2)) + QubitOperator(R_op, 1j*np.sin(angle/2)) 
    R.append(qOp)
#     R+= qOp

# A = reduce(lambda x,y : x+y, Script_A)

In [ ]:
# R* A * hermitian_conjugated(R)

In [ ]:
from copy import deepcopy

new_A = deepcopy(Script_A)
for rot in R:
    new_A = [rot *A_op for A_op in new_A]
    


In [ ]:
# dict(zip(AC_set,ep_state))

In [ ]:
AA = []

for term, coeff in zip(AC_set,ep_state):
    Pword = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(term) if Pstr!='I'])
    op = QubitOperator(Pword,coeff)
    AA.append(op)
    
AA

In [ ]:
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import Get_Xsk_op_list
S_index = 0
N_Qubits=3
check_reduction=True

X_sk_theta_sk_list, normalised_FULL_set, Ps, gamma_l = Get_Xsk_op_list(AA, 
                                                                       S_index, 
                                                                       N_Qubits,
                                                                       check_reduction=True,
                                                                       atol=1e-8,
                                                                       rtol=1e-05)

R_sk_OP_list = []
for X_sk_Op, theta_sk in X_sk_theta_sk_list:
    op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
    R_sk_OP_list.append(op)

R_S_op = reduce(lambda x,y: x*y, R_sk_OP_list[::-1])  # <- note reverse order!

from openfermion import hermitian_conjugated
R_S_dag = hermitian_conjugated(R_S_op)

In [ ]:
R_S_op * reduce(lambda x,y: x+y, AA) * R_S_dag

In [ ]:
ham_rotated = dict(zip(AC_set,ep_state))
ham_rotated

In [ ]:
r = rotations[0]

ham_next = {}
for t in ham_rotated.keys():
    t_set_next = c.apply_rotation(r,t)
    print(t_set_next)
    print(ham_rotated[t])
    print()
    for t_next in t_set_next.keys():
        if t_next in ham_next.keys():
            ham_next[t_next] = ham_next[t_next] + t_set_next[t_next]*ham_rotated[t]
        else:
            ham_next[t_next] = t_set_next[t_next]*ham_rotated[t]
    
ham_next

In [ ]:
0.6324555320336759 * -0.5477225575051661

In [ ]:
rot_H = deepcopy(Script_A)
print(rot_H)

R0 = R[0]
R0

In [ ]:
h_next =QubitOperator()
for t in rot_H:
#     next_t = R0* QubitOperator(*list(t.terms.keys()), 1) * hermitian_conjugated(R0)
    next_t = R0*t #* hermitian_conjugated(R0)
    print(list(next_t))
    print('##')
    h_next+=next_t
    print(h_next)
    print('t:', t)
    print()
    print()

h_next

In [ ]:
R0*reduce(lambda x,y: x+y, rot_H) * hermitian_conjugated(R0)

In [ ]:
R0*QubitOperator(*list(Script_A[0].terms.keys()), 1)#*hermitian_conjugated(R0)

In [ ]:
Script_A[0]

In [ ]:
r = rotations[0]
t = list(ham_rotated.keys())[0]
t_set_next = c.apply_rotation(r,t)
t_set_next

In [ ]:
index = 2
t = list(ham_rotated.keys())[index]
t_set_next = c.apply_rotation(r,t)
t_set_next

In [ ]:
R0* QubitOperator(*list(Script_A[index].terms.keys()), 1)*hermitian_conjugated(R0)

In [ ]:
1j*-1j

In [ ]:
theta = np.pi/3

np.cos(theta)**2 + np.sin(theta)**2

In [ ]:
def apply_rotation_NEW(rotation,p):
    """
    Performs ( Rotation @ P @ rotation^{†} ) = out
    
    """
    
    out = {}
    
    if not c.commute(rotation[1],p):
        if rotation[0] == 'pi/2':
#             q = pauli_mult(rotation[1],p)
#             out[q[0]] = (1j*q[1]).real
            out[p] = -1.0
    
        else:
#             out[p] = np.cos(rotation[0])
            out[p] =  np.cos(rotation[0])**2 - np.sin(rotation[0])**2
    
            q = c.pauli_mult(rotation[1],p)
#             out[q[0]] = (1j*q[1]*np.sin(rotation[0])).real
            out[q[0]] = 2j*np.cos(rotation[0])*np.sin(rotation[0])*q[1]
            
    else:
            out[p] = 1.
    
    return out

In [ ]:
def apply_rotation_NEW_left(rotation,p):
    """
    Performs ( Rotation @ P ) = out
    
    """
    
    out = {}
    
    if not c.commute(rotation[1],p):
        if rotation[0] == 'pi/2':
            q = c.pauli_mult(rotation[1],p)
            out[q[0]] = (1j*q[1])

        else:
            out[p] =  np.cos(rotation[0])
            q = c.pauli_mult(rotation[1],p)
            out[q[0]] = (1j*q[1]*np.sin(rotation[0])).real           
    else:
        q = c.pauli_mult(rotation[1],p)
        if rotation[0] == 'pi/2':
            out[q[0]] = (1j*q[1])
        else:
            out[p] =np.cos(rotation[0])
            out[q[0]] = (1j*q[1]*np.sin(rotation[0]))
    
    return out

In [ ]:
R0

In [ ]:
r

In [ ]:
# commuting case!
IXX_str = 'IXX'
out = c.apply_rotation(r,IXX_str)
out

In [ ]:
IXX = QubitOperator('X1 X2', 1)
R0*IXX

In [ ]:
# anti-commuting case!
IXY_str = 'IXY'
out = c.apply_rotation(r,IXY_str)
out

In [ ]:
IXY = QubitOperator(*list(Script_A[1].terms.keys()), 1)

In [ ]:
R0*IXY#*hermitian_conjugated(R0)

In [ ]:
R0*IXY*hermitian_conjugated(R0)

In [ ]:
# commuting case!
XZI_str = 'XZI'
out = c.apply_rotation(r,XZI_str)
out

In [ ]:
IXX = QubitOperator('X0 Z1', 1)
R0*IXX

In [ ]:
R0*IXX*hermitian_conjugated(R0)

In [ ]:
apply_rotation_NEW_left(r,XZI_str)

In [ ]:
IXX = QubitOperator(*list(Script_A[0].terms.keys()), 1)


In [ ]:
t = list(ham_rotated.keys())[0]
t

In [ ]:
t = list(ham_rotated.keys())[0]
t_set_next = apply_rotation_NEW(r,t)
t_set_next

In [ ]:
t = list(ham_rotated.keys())[0]
t_set_next = c.apply_rotation(r,t)
t_set_next

In [ ]:
t = list(ham_rotated.keys())[0]
t_set_next_left = apply_rotation_NEW_left(r,t)
t_set_next_left

In [ ]:
R0*IXX

In [ ]:
t = list(ham_rotated.keys())[1]
t

In [ ]:
XZI = QubitOperator(*list(Script_A[3].terms.keys()), 1)

In [ ]:
R0*XZI*hermitian_conjugated(R0)

In [ ]:
t = list(ham_rotated.keys())[1]
t_set_next = apply_rotation_NEW(r,t)
t_set_next

In [ ]:
t_set_next = c.apply_rotation(r,t)
t_set_next

In [ ]:
R0*QubitOperator(*list(Script_A[0].terms.keys()), 1)*hermitian_conjugated(R0)

In [ ]:
R0

In [ ]:
q = c.pauli_mult(r[1],t)
q

In [ ]:
q = c.pauli_mult(r_new[1],t)
q[1]*1j

In [ ]:
r_new = deepcopy(r)
r_new[0] = 'pi/2'

t = list(ham_rotated.keys())[1]
t_set_next = c.apply_rotation(r_new,t)
t_set_next

In [ ]:
QubitOperator(*list(Script_A[3].terms.keys()), 1)

In [ ]:
t

In [ ]:
apply_rotation_NEW(r_new,t)

In [ ]:
apply_rotation_NEW_left(r_new,t)

In [ ]:
r_new[0] = np.pi/2
R_op_str = ' '.join(['{}{}'.format(Pstr, qNo) for qNo, Pstr in enumerate(r_new[1]) if Pstr!='I'])
R_new = QubitOperator('', np.cos(r_new[0])) + QubitOperator(R_op_str, 1j*np.sin(r_new[0])) 

In [ ]:
R_new* QubitOperator(*list(Script_A[3].terms.keys()), 1)#*hermitian_conjugated(R_new)

In [ ]:
-1* QubitOperator(*list(Script_A[1].terms.keys()), 1)

$$ U = e^{+i \theta R} = \cos(\theta)\mathcal{I} + i\sin(\theta)R$$

- if $R$ and $P$ anticommute
$$U P U^{\dagger} = \big(\cos^{2}(\theta) - \sin^{2}(\theta)\big) P + 2i\cos(\theta)\sin(\theta)RP $$

- else if $R$ and $P$ commute
$$U P U^{\dagger} =\cos^{2}(\theta) P + \sin^{2}(\theta)P = P $$


- left hand only:

$$U P = \cos(\theta)P + i\sin(\theta)RP$$



- overall:
    - if $\theta = \pi/2$ then
        - $U P U^{\dagger} = -P$ (a.c. case)
        - $U P U^{\dagger} = +P$ (commuting case)



- left hand only:

$$U P = \cos(\theta)P - i\sin(\theta)RP$$



In [ ]:
def Openfermion_QubitOp_to_op_str(QubitOp, N_qubits):

    PauliStr, coeff = tuple(*QubitOp.terms.items())
    P_list = np.array(['I' for _ in range(N_qubits)], dtype=object)
    if PauliStr:
        qNo_list, qPstr_list = zip(*PauliStr)
        P_list[list(qNo_list)] = qPstr_list

    op_str = ''.join(P_list.tolist())
    return op_str, coeff

In [ ]:
Openfermion_QubitOp_to_op_str(QubitOperator('', 2), 5)

In [ ]:
X = np.zeros(4)

X[[1,2]]=(1,3)
X.tolist()

In [ ]:
from quchem.Misc_functions.conversion_scripts import Get_Openfermion_Hamiltonian

H_full = Get_Openfermion_Hamiltonian(ham)

In [ ]:
rotations = []
ep_state = ground_state_params
# if there are cliques...
if fn_form[1] > 0:
    # rotations to map A to a single Pauli (to be applied on left)
    for i in range(1,fn_form[1]):
        theta = np.arctan2(ep_state[1][i],np.sqrt(sum([ep_state[1][j]**2 for j in range(i)])))
        if i == 1 and ep_state[1][0] < 0:
            theta = np.pi - theta
        generator = c.pauli_mult(model[1][0],model[1][i])
        sgn = generator[1].imag
        rotations.append( [sgn*theta, generator[0]] )
rotations

In [ ]:
AC_set = list(Get_Openfermion_Hamiltonian(dict(zip(Ci1_list, ep_state[1]))))
AC_set

In [ ]:
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import Get_Xsk_op_list
S_index = 0
N_Qubits=3
check_reduction=True

X_sk_theta_sk_list, normalised_FULL_set, Ps, gamma_l = Get_Xsk_op_list(AC_set, 
                                                                       S_index, 
                                                                       N_Qubits,
                                                                       check_reduction=True,
                                                                       atol=1e-8,
                                                                       rtol=1e-05)

R_sk_OP_list = []
for X_sk_Op, theta_sk in X_sk_theta_sk_list:
    op = np.cos(theta_sk / 2) * QubitOperator('') -1j*np.sin(theta_sk / 2) * X_sk_Op
    R_sk_OP_list.append(op)

R_S_op = reduce(lambda x,y: x*y, R_sk_OP_list[::-1])  # <- note reverse order!



In [ ]:
from openfermion import hermitian_conjugated

my_out = R_S_op * H_full * hermitian_conjugated(R_S_op)

In [ ]:
ham_rotated = deepcopy(ham)

for r in rotations: # rotate the full Hamiltonian to the basis with diagonal noncontextual generators
    ham_next = {}
    for t in ham_rotated.keys():
        t_set_next = c.apply_rotation(r,t)
#         print(t_set_next)
        for t_next in t_set_next.keys():
            if t_next in ham_next.keys():
                ham_next[t_next] = ham_next[t_next] + t_set_next[t_next]*ham_rotated[t]
            else:
                ham_next[t_next] = t_set_next[t_next]*ham_rotated[t]
    ham_rotated = deepcopy(ham_next)

old_out = Get_Openfermion_Hamiltonian(ham_rotated)

In [ ]:
my_out == old_out